In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import sympy # equation solver
from scipy.optimize import optimize # minimizing
# import statsmodels.api as sm # qqplot
import h5py # lese matlab data
# h5py-3.11.0
%matplotlib inline
#
# quantile-quantile plot
# import statsmodels.api as sm
#sm.qqplot_2samples(x, y, line='45')
#plt.title('Q-Q plot of x and y')
#plt.xlabel('Quantiles of x')
#plt.ylabel('Quantiles of y')
#plt.grid(True)
#plt.show()

# Constants, from Jenkins
rho_i=916
L_i=334000
rho_ic_i_k_i=2.1
rho_w=1030
c_w=3974
c_i=2108 # from internet, not in Jenkins tables

gamma_1=-0.0573
gamma_2=0.0832
gamma_3=-7.53*10**-4

# Elins' values
# CdTt=0.0011 # Thermal Stanton
# CdTs=3.1*10**-5 # Diffusion Stanton number
# CdTts=5.9*10**-4 # Stanton number - not used?

#CdTt=0.02*0.0011 # Thermal Stanton
#CdTs=0.15*3.1*10**-5 # Diffusion Stanton number

# best fit so far
# CdTt=0.15*0.0011 # Thermal Stanton
# CdTs=0.2*3.1*10**-5 # Diffusion Stanton number
# r2 = 0.65 for Pb O 400

CdTt=0.15*0.0011 # Thermal Stanton
CdTs=0.3*3.1*10**-5 # Diffusion Stanton number

CdTts=5.9*10**-4 # Stanton number - not used?
Cd=0.0097 # not used
Tt=0.011 # not used
Ts=3.1*10**-4 # not used
Tts=0.006 #  not used

# Fixed values
T_i=-20
S_w=34.4
Pb=360


# Constants, Elin
A=rho_i*L_i
B=rho_i*c_i*T_i #
C=rho_i*c_i  # 
D=rho_w*c_w*CdTt
E=rho_w*CdTs
F=gamma_2+gamma_3*Pb

# a is melt/second! multiply by numbers of seconds per year to get m/yr
N=365*24*3600

fig,ax=plt.subplots()

for U in np.linspace(0,0.12,5):
    TT=np.linspace(-1.95,-1,25)
    aa=[]
    for i,T in enumerate(TT):
        a = sympy.Symbol('a')
        f1=(B-A)*a-C*a*(gamma_1*S_w*E/(E+rho_i*a)+F)-D*U*(gamma_1*S_w*E/(E+rho_i*a)+F)+D*U*T
        aa.append(N*sympy.nsolve(f1,a,1/N))
    ax.plot(TT,aa,label='U='+str(U)+'m/s')
ax.legend()
ax.grid()   
ax.set_ylabel('Melt rate [m/yr]')


fig,ax=plt.subplots()
for T in [-1.9,-1.7,-1.5]:
    UU=np.linspace(0.01,0.12,12)
    aa=[]
    for i,U in enumerate(UU):
        a = sympy.Symbol('a')
        f1=(B-A)*a-C*a*(gamma_1*S_w*E/(E+rho_i*a)+F)-D*U*(gamma_1*S_w*E/(E+rho_i*a)+F)+D*U*T
        aa.append(N*sympy.nsolve(f1,a,1/N))
    ax.plot(UU,aa,label='T='+str(T)+'C')
ax.legend()
ax.grid()   
ax.set_ylabel('Melt rate [m/yr]')


KeyboardInterrupt: 

In [ ]:

U=0.1
T=-1.9
a = sympy.Symbol('a')
f1=(B-A)*a-C*a*(gamma_1*S_w*E/(E+rho_i*a)+F)-D*U*(gamma_1*S_w*E/(E+rho_i*a)+F)+D*U*T
a = sympy.nsolve(f1,a,1/N)

S_b=S_w*E/(E+rho_i*a)
T_b=gamma_1*S_b+F

check=-A*a+B*a-C*T_b*a-D*U*(T_b-T)
print(check) # Should be very close to zero
print('S_b='+str(S_b))
print('T_b='+str(T_b))
print('Meltrate='+str(a*N))

In [ ]:
aa=rho_i*(B-A-C*F)
bb=E*(B-A)-C*gamma_1*S_w*E-C*F*E-D*U*F*rho_i+D*U*T*rho_i
cc=D*U*T*E-D*U*F*E-D*U*gamma_1*S_w*E

a=(-bb-np.sqrt(bb**2-4*aa*cc))/(2*aa) # meltrate [m/s]
print(str(a*N))

Try to implement minimization of RMS of observed melt and parameterized  melt based from 3-eqn solver following this recepie: https://datascience.stackexchange.com/questions/69092/how-to-minimize-mean-square-error-using-python

In [ ]:
# read obs data
f = h5py.File('M2_series.mat','r')
print(f.keys())

num = np.array(f.get('num'))
dh = np.array(f.get('dh'))
vsr = np.array(f.get('vsr'))
vsr_lin = np.array(f.get('vsr_lin'))
t2 = np.array(f.get('t2'))
spd2 = np.array(f.get('spd2'))

# tr2 = np.array(f.get('t2'))
spdf2 = np.array(f.get('spdf2'))

# mm = -(dh-np.mean(vsr,axis=0))
mm = -(dh-np.mean(vsr,axis=0))-0.45 # offset of staek strain
# mm = -(dh-vsr_lin)-0.45 # offset of staek strain
t = t2.T
spd = spd2.T
spd_ = spdf2.T

print('langley dh = ')
print(str(-(0.9+0.84)))

print('ApRES dh = ')
print(str(np.nanmean(dh)))

print('stake strain = ')
print(str(0.9))
print('ApRES strain = ')
print(str(np.mean(vsr,axis=0)))

fig,ax=plt.subplots()
ax.plot(num, mm)
fig,ax=plt.subplots()
ax.plot(num, t)
fig,ax=plt.subplots()
ax.plot(num, spd)
ax.plot(num, spd_)



In [ ]:
# find non-neagtive values
ii = np.isfinite(spd * t * mm) # & (spd>0.05)

UU = spd[ii] # + 0.07 fits magntude, but worsens r2
TT = t[ii] # + 0.15
mm_obs = mm[ii]


#rnd = np.random.rand(100)
#rndi = np.round(rnd*len(UU))

#UU = UU[rndi.astype('int64')]
#TT = TT[rndi.astype('int64')]
#mm_obs = mm_obs[rndi.astype('int64')]

#print(str(mm_obs))

In [ ]:
# velocity and temperature series
#UU = np.linspace(0,0.12,25)
#TT = np.linspace(-1.95,-1,25)
## melt rate series
#melt_obs = UU*TT

# find non-neagtive values
# ii = np.isfinite(spd * t * mm)
# UU = spd[ii] # + 0.07 fits magntude, but worsens r2
# TT = t[ii] # + 0.15
# mm_obs = mm[ii]

# Fixed values
T_i=0 #-20
S_w=34.4
Pb=360

def melt_para(ftt,fts,toff,uoff):
    aa=[]
    # CdTt=0.15*0.0011 # Thermal Stanton
    # CdTs=0.2*3.1*10**-5 # Diffusion Stanton number
    CdTt=ftt*0.0011 # Thermal Stanton
    # CdTs=fts*3.1*10**-5 # Diffusion Stanton number
    CdTs=CdTt/30

    # Constants, Elin
    A=rho_i*L_i
    B=rho_i*c_i*T_i #
    C=rho_i*c_i  # 
    D=rho_w*c_w*CdTt
    E=rho_w*CdTs
    F=gamma_2+gamma_3*Pb

    T = TT + toff
    U = UU + uoff
    aa=rho_i*(B-A-C*F)
    bb=E*(B-A)-C*gamma_1*S_w*E-C*F*E-D*U*F*rho_i+D*U*T*rho_i
    cc=D*U*T*E-D*U*F*E-D*U*gamma_1*S_w*E
    aa=N*(-bb-np.sqrt(bb**2-4*aa*cc))/(2*aa) # meltrate [m/yr]
    # add convective part
    # aa[U<0.02] = aa[U<0.02] +1.93*np.abs(T[U<0.02]+2.15)**(4/3)
    # aa[U<0.02] = aa[U<0.02] +2.5*np.abs(T[U<0.02]+2.15)**(4/3)
    # aa = aa +5.5*np.abs(T+2.15)**(4/3)
    # aa = aa +2*np.abs(T+2.15)**(4/3)
    aa = aa +fts*np.abs(T+2.15)**(4/3)
    
    #for i,element in enumerate(TT):
    #    T = TT[i] + toff
    #    U = UU[i] + uoff
    #    #a = 5 +5
    #    # a = T_i +5
    #    a = sympy.Symbol('a')
    #    f1=(B-A)*a-C*a*(gamma_1*S_w*E/(E+rho_i*a)+F)-D*U*(gamma_1*S_w*E/(E+rho_i*a)+F)+D*U*T
    #    aa.append(N*sympy.nsolve(f1,a,1/N))
    return aa


# from sklearn.metrics import r2_score

def r2_score_man(y_obs, y_pred):
    """
    Calculate the coefficient of determination (R^2) between the observed data and the predicted data.

    Parameters:
    y_obs (numpy array): The observed data.
    y_pred (numpy array): The predicted data.

    Returns:
    float: The R^2 value.
    """
    # Calculate the residuals (differences between observed and predicted data)
    resid = y_pred - y_obs
    
    # Calculate the sum of squares of residuals
    SSresid = np.sum(resid ** 2)
    
    # Calculate the total sum of squares (variance of the observed data times the number of observations minus one)
    # Note: np.var with ddof=1 calculates the sample variance
    SStotal = (len(y_obs) - 1) * np.var(y_obs, ddof=1)
    
    # Calculate the R^2 value
    r2_val = 1 - SSresid / SStotal
    
    return r2_val

# Example usage
# ys = np.array([...])  # Observed data
# y1s = np.array([...]) # Predicted data
# r_squared = calculate_r_squared(ys, y1s)
# print(r_squared)

# mm_par = melt_para(ftt,fts,toff,soff)
# mm_par = melt_para(0.15,0.2,0,0)

# first extension with so far best fit
#ftt = np.arange(0.01, 0.1, 0.005) # np.array([0.01, 0.05, 0.15, 0.2, 0.5, 1, 1.5])
#fts = np.array([0.005, 0.007, 0.01, 0.013, 0.015, 0.4, 0.5, 0.6]) # np.arange(0.01, 1.2, 0.01) # np.array([0.01, 0.05, 0.15, 0.2, 0.5, 1, 1.5])
#toffs = np.arange(-0.15, 0.15, 0.01) 
#uoffs = np.arange(-0.02, 0.08, 0.01) # np.arange(-0.1, 0.1, 0.01) 



ftt = np.arange(0.01, 0.1, 0.005) # np.array([0.01, 0.05, 0.15, 0.2, 0.5, 1, 1.5])
fts = np.array([0.005, 0.007, 0.01, 0.013, 0.015, 0.4, 0.5, 0.6, 1.5]) # np.arange(0.01, 1.2, 0.01) # np.array([0.01, 0.05, 0.15, 0.2, 0.5, 1, 1.5])
toffs = np.arange(-0.15, 0.15, 0.01) 
uoffs = np.arange(-0.02, 0.08, 0.01) # np.arange(-0.1, 0.1, 0.01) 

# large fts values
#ftt = np.arange(0.01, 0.2, 0.005) # np.array([0.01, 0.05, 0.15, 0.2, 0.5, 1, 1.5])
#fts = np.array([0.01, 0.1, 1, 1.5, 10, 10^5, 10**6, 5*10**7, 10**8, 5*10**8, 10**10, 10**15]) # np.arange(0.01, 1.2, 0.01) # np.array([0.01, 0.05, 0.15, 0.2, 0.5, 1, 1.5])
#toffs = np.arange(-0.16, 0.16, 0.02) 
#offs = np.arange(-0.02, 0.08, 0.01) # np.arange(-0.1, 0.1, 0.01) 

# original small set with good fit for offset, explore large fts gives best fit with square of filtered velocities
ftt = np.array([0.09, 0.15, 1, 1.5, 15, 150]) # np.arange(0.05, 0.15, 0.01)
fts = np.array([0.2, 0.25, 0.3, 0.35, 1, 10**9, 5*10**9, 10**10, 5*10**10, 10**11])
toffs = np.arange(-0.05, 0.15, 0.01) 
uoffs = np.arange(-0.05, 0.1, 0.01) 


# explor optiosn to add convective mixing
#ftt = np.array([1, 2, 10])
ftt = np.arange(0.01, 0.15, 0.02) # np.array([0.01, 0.05, 0.09, 0.15, 0.2, 0.5, 0.7, 1, 1.5]) # np.arange(0.05, 0.15, 0.01)
# ftt = np.arange(0.1, 1.5, 0.1)
#fts = np.array([0.2, 0.25, 0.3, 0.35, 1, 10**9, 5*10**9, 10**10, 5*10**10, 10**11])
#fts = np.array([0.1, 0.5, 1, 10])
#fts = np.array([1])
#fts = np.arange(20, 40, 2) # np.array([25, 30, 35, 50])
fts = np.array([0, 2, 4])
toffs = np.arange(-0.05, 0.15, 0.01) 
uoffs = np.arange(-0.05, 0.1, 0.01) 


# original small set with good fit for offset
#ftt = np.array([0.01, 0.05, 0.15, 0.2, 0.5, 1, 1.5])
#fts = np.array([0.01, 0.05, 0.15, 0.2, 0.5, 1, 1.5])
#toffs = np.arange(-0.15, 0.15, 0.01) 
#uoffs = np.arange(-0.1, 0.1, 0.01) 

print('initialize arrays')

mm_pars = np.zeros((len(ftt), len(fts), len(toffs), len(uoffs), len(ii)))*np.nan
mm_r2s_off = np.zeros((len(ftt), len(fts), len(toffs), len(uoffs)))
mm_r2s_abs = np.zeros((len(ftt), len(fts), len(toffs), len(uoffs)))

print('start loop')

for i in range(ftt.size):
    #print(str(i))
    for j in range(fts.size):
        for k in range(toffs.size):
            for l in range(uoffs.size):
                dum = np.array(melt_para(ftt[i],fts[j],toffs[k],uoffs[l]))
                
                mm_pars[i, j, k, l, np.squeeze(ii)] = dum
                mm_r2s_off[i, j, k, l] = r2_score_man(mm_obs-np.mean(mm_obs), dum-np.mean(dum))
                mm_r2s_abs[i, j, k, l] = r2_score_man(mm_obs, dum)
    sys.stdout.write('.')
                #print(str(i))
                #print(str(j))
                #print(str(k))
                #print(str(l))    
    
print('done')


In [ ]:
#mm_r2s
im_aoff = np.where(mm_r2s_off==np.nanmax(mm_r2s_off[:,:,:,:])) # force toff=0, uoff=0, allow free abs offset
im_off = np.where(mm_r2s_off==np.nanmax(mm_r2s_off[:,1,5,5])) # force toff=0, uoff=0, allow free abs offset
#im_off = np.where(mm_r2s_off==np.nanmax(mm_r2s_off)) # allow free uoff
im_abs = np.where(mm_r2s_abs==np.nanmax(mm_r2s_abs[:,:,:,:])) # allow free T & U
# im_abs = np.where(mm_r2s_off==np.nanmax(mm_r2s_off[:,:,5,:])) # force toff=0

print(str(np.nanmax(mm_r2s_off)))
print(im_aoff)
print(str(np.nanmax(mm_r2s_off[im_off])))
print(im_off)
print(str(np.nanmax(mm_r2s_abs[im_abs])))
print(im_abs)



print('')
print(ftt[im_aoff[0]])
print(ftt[im_off[0]])
print(ftt[im_abs[0]])
print('')
print(fts[im_aoff[1]])
print(fts[im_off[1]])
print(fts[im_abs[1]])
print('')
print(toffs[im_aoff[2]])
print(toffs[im_off[2]])
print(toffs[im_abs[2]])
print('')
print(uoffs[im_aoff[3]])
print(uoffs[im_off[3]])
print(uoffs[im_abs[3]])


original melt rates, manual optimization

0.708350356043953
(array([1], dtype=int64), array([9], dtype=int64), array([5], dtype=int64), array([7], dtype=int64))
0.702708877358194
(array([0], dtype=int64), array([3], dtype=int64), array([17], dtype=int64), array([12], dtype=int64))

[0.15]
[0.09]

[1.e+11]
[0.35]

[6.9388939e-18]
[0.12]

[0.02]
[0.07]

0.5 m/yr offset on melt rates

0.708350356043953
(array([1], dtype=int64), array([9], dtype=int64), array([5], dtype=int64), array([7], dtype=int64))
0.7070218038073472
(array([1], dtype=int64), array([4], dtype=int64), array([6], dtype=int64), array([7], dtype=int64))

[0.15]
[0.15]

[1.e+11]
[1.]

[6.9388939e-18]
[0.01]

[0.02]
[0.02]

In [ ]:
# 2 1 6 3
#mmp=mm_pars[16,7,27,8,:]
# mmp=mm_pars[12,5,28,9,:]
# mmp = mm_pars[im_off].T
mmp = mm_pars[im_off[0], im_abs[1], im_off[2], im_off[3]].T
mmpa = mm_pars[im_aoff[0], im_aoff[1],im_aoff[2], im_aoff[3]].T

mmp1 = mmp-(np.nanmean(mmp)-np.nanmean(mm))
mmp10 = mmpa-(np.nanmean(mmpa)-np.nanmean(mm))
# mmp1=mm_pars[2,1,5,7,:]
# mmp2=mm_pars[15,7,28,9,:]
mmp2 = mm_pars[im_abs[0], im_abs[1], im_abs[2], im_abs[3]].T

print('')
print(str(np.nanmean(mmp)-np.nanmean(mm)))
print(str(np.nanmean(mmpa)-np.nanmean(mm)))

fig,ax=plt.subplots(2)
# is more concise than this:
# fig = plt.figure()
# ax = fig.add_subplot(111)

# fig.figure(figsize=(10,6))
fig.set_figwidth(20)
fig.set_figheight(12)

ax[0].plot(num, mm)
ax[0].plot(np.squeeze(num), np.squeeze(mmp1))
#ax[0].plot(np.squeeze(num), np.squeeze(mmp10))
#ax.plot(np.squeeze(num), np.mean(mmp[np.squeeze(ii)]))
ax[0].set_title('melt offset corrected')

ax[1].plot(num, mm)
ax[1].plot(np.squeeze(num), np.squeeze(mmp2))
ax[1].set_title('Temperature and velocity offset corrected')

plt.show()
#plt.savefig("mygraph.png")

#print(str(np.corrcoef(mm[ii],mmp[np.squeeze(ii)])))

#print(str(r2_score_man(mm[ii]-np.mean(mm[ii]),mmp[np.squeeze(ii)]-np.mean(mmp[np.squeeze(ii)]))))

#%matplotlib notebook
#%matplotlib inline
#%matplotlib qt

fig,ax=plt.subplots(1,2)
fig.set_figwidth(10)
ax[0].plot(mmp, mm,'.')
ax[0].plot([0, 4],[0, 4])

ax[1].plot(mmp2, mm,'.')
ax[1].plot([0, 4],[0, 4])

plt.show()



fig,ax=plt.subplots(3,2)
fig.set_figwidth(10)
fig.set_figheight(10)
ax[0,0].plot(spd*(2.15+t), mmp1,'.')
ax[0,1].plot(spd*(2.15+t), mmp2,'.')
ax[1,0].plot(np.sqrt(spd)*(2.15+t), mmp1,'.')
ax[1,1].plot(np.sqrt(spd)*(2.15+t), mmp2,'.')
ax[2,0].plot((spd)*(2.15+t)*(2.15+t), mmp1,'.')
ax[2,1].plot((spd)*(2.15+t)*(2.15+t), mmp2,'.')
plt.show()


fig,ax=plt.subplots(1,2)
fig.set_figwidth(10)
fig.set_figheight(5)

ii = np.isfinite(mm * mmp1 * mmp2)

qts = np.arange(0.001, 0.999, 0.001) 
qmm = np.quantile(mm[ii],qts)
qmmp1 = np.quantile(mmp1[ii],qts)
qmmp2 = np.quantile(mmp2[ii],qts)


ax[0].plot(qmmp1, qmm,'.')
ax[0].plot([0, 4], [0, 4])

ax[1].plot(qmmp2, qmm,'.')
ax[1].plot([0, 4], [0, 4])

plt.show()



In [ ]:
dum = np.zeros(np.shape(dum))
print(dum)

In [ ]:
from scipy.optimize import minimize # minimizing

def f(params):
    dum = np.array(melt_para(params[0],params[1],params[2],params[3]))
    if np.isnan(dum.any()):
        print('nan encountered')
        dum = np.zeros(np.shape(dum))
    return 1-r2_score_man(mm_obs, dum)

opti_arr = [ftt[im_abs[0]], fts[im_abs[1]], toffs[im_abs[2]], uoffs[im_abs[3]]]
#opti_arr = [ftt[im_abs[0]], fts[im_abs[1]], [0], [0]]
opti = np.squeeze(np.array(opti_arr).T)
print(opti)
r2 = f(opti)
print(r2)
print(mm_r2s_abs[im_abs])

initial_guess = opti #[0.1, 10, 0, 0.2] # opti
# result = minimize(f, initial_guess, method='Powell') # solves well for melt rate offset
# result = minimize(f, initial_guess, method='CG', options={'maxiter':2000}) % solves with precisison loss for full melt rate
result = minimize(f, initial_guess, method='BFGS')

print(result)

m_ = melt_para(ftt[im_abs[0]], fts[im_abs[1]], toffs[im_abs[2]], uoffs[im_abs[3]])
ii = np.argwhere(np.isnan(m_))
print(np.max(m_))
print(ii)

In [ ]:

print(' ')
print(['GammaT *', str(ftt[2])])
print(['GammaS *', str(fts[6])])
print(['T offset ', str(toffs[3])])
print(['ustar offset ', str(uoffs[1])])

In [ ]:
np.shape(dum)

In [ ]:
fig,ax=plt.subplots(2,1)
fig.set_figwidth(10)
ax[0].plot(num,spd)
ax[1].plot(num,t)

relation of melt rates to ocean properties.
At zero order, ice ablation increases with larger ambient ocean temperature and velocity as both factors enhance the fluxes across the ice-ocean boundary layer. From the conservation of heat and salt, a quadratic equation for the ablation rate can be found that generally yields a non-linear response to changes in ambient thermal driving T* and friction velocity u* = C_d^(1/2)*U_0, while a linearized expression is obtained under the assumption of small variations of the salinity in the boundary layer.

If shear-driven turbulence from ambient currents was the primary source of mixing across the IOBL, the ablation rate would decrease to zero when the free-stream velocity U_0 vanishes. However, recent studies showed boundary layer turbulence is caused  by natural convection (buoyancy- or shear-driven) beneath a sloping ice face that leads to finite ablation rates in absence of ambient currents (find better wording here). These DNS simulations suggest 

while other processes have been proposed to . However, recent studies have shown that intrinsic instabilities in the convectively driven turbulence in 


Assessment of melt parameterizations.

The observed melt rates can be parameterized. Several fits with different parameters. 3 eqn, 2 eqn, emprirical relationship.
Varying strain would indicate a decreasing trend in melt rate that is not supported by the ocean observations. While the 3eqn. parameterization captures a large fraction of the variability, explanations for the offsets in u and t and the linear strain are less obvious. The 3 eqn fit suggests an offset in the observed velocities of 4 cm/s, which leads to an underestimate of the melt rates for low velocities. This may be related to buoyant plumes that are rising along the sloping ice base and dominate the velocity structure at the ice-ocean interface under low background flow conditions. If the linear trend in the strain is related the ice dynamics of the basal channel, this may also affect the slope and hence the plume velocity, providing an explanation for a trend in melt rates that woudl not be observed as changes in ocean properties 30-40 m below the ice base where our mooring is situated. The offset in temperature/ thermal driving is still hard to explain (needing a temperature inversion below the ice base), casting doubt on the absolute magnitude of the observed (parameterized) melting (and strain).

Stanton numbers for:
3eqn with melt offset, 3eqn with t&u offset, 2eqn same way; derive Cd range, gammat/gammas ratio; compare fit & distributions; project melt rates back with unceratainty

In [ ]:
from scipy import stats

In [ ]:
cmap = plt.cm.get_cmap('Spectral_r', 64)
dt = t+2.15 # thermal driving
bool = ~np.isnan(dt) & ~np.isnan(spd) & ~np.isnan(mm) # fint finite value tupels

dtbins = np.arange(0, 0.35, 0.01)
ubins = np.arange(-0.02, 0.2, 0.005)
mbins  = np.arange(0, 4, 0.05)
utbins  = np.arange(-0.001, 0.04, 0.0005)

hist = stats.binned_statistic_2d( spd[bool], dt[bool], None, bins=[ubins, dtbins], statistic='count')  # Returns a object
hist.statistic[hist.statistic == 0] = np.nan  # Sets all values of 0 to nan as log10(0) = -inf

#In using stats.binned_statistic_2d, the input argument needs to be a 1-D array.  If you're using higher-
#dimensional data, such as model data, then you would want to flatten the input arrays with np.ravel(). 
#Similarly the input arguments should contain no NaNs.  If they do, you would want to remove then with
#bool = ~np.isnan(x) & ~np.isnan(y)
#hist = stats.binned_statistic_2d(x[bool], y[bool], None, bins=[xbins,ybins], statistic='mean').statistic  

mmean = stats.binned_statistic_2d( spd[bool], dt[bool], mm[bool], bins=[ubins, dtbins], statistic='mean')  # Returns a object

# melt rate space
muhist = stats.binned_statistic_2d( spd[bool], mm[bool], None, bins=[ubins, mbins], statistic='count')  # Returns a object
muhist.statistic[muhist.statistic == 0] = np.nan  # Sets all values of 0 to nan as log10(0) = -inf
mumean = stats.binned_statistic_2d( spd[bool], mm[bool], dt[bool], bins=[ubins, mbins], statistic='mean')  # Returns a object


mthist = stats.binned_statistic_2d( dt[bool], mm[bool], None, bins=[dtbins, mbins], statistic='count')  # Returns a object
mthist.statistic[mthist.statistic == 0] = np.nan  # Sets all values of 0 to nan as log10(0) = -inf
mtmean = stats.binned_statistic_2d( dt[bool], mm[bool], spd[bool], bins=[dtbins, mbins], statistic='mean')  # Returns a object

# u*t space

uthist = stats.binned_statistic_2d( dt[bool]*spd[bool], mm[bool], None, bins=[utbins, mbins], statistic='count')  # Returns a object
uthist.statistic[uthist.statistic == 0] = np.nan  # Sets all values of 0 to nan as log10(0) = -inf
uttmean = stats.binned_statistic_2d( dt[bool]*spd[bool], mm[bool], dt[bool], bins=[utbins, mbins], statistic='mean')  # Returns a object

#################### plotting

fig, ax = plt.subplots(1, 2,figsize=np.array([18, 10]))

plt.sca(ax[0]) 
image = plt.pcolormesh(ubins, dtbins, np.log10(hist.statistic.T), cmap=cmap, shading='flat') 
plt.title('Distribution of u-dt pairs')

plt.sca(ax[1]) 
image = plt.pcolormesh(ubins, dtbins, mmean.statistic.T, cmap=cmap, shading='flat') 
plt.title('Mean melt rate')

plt.subplots_adjust(wspace=0.025)
fig.colorbar(image, ax=ax[0], orientation='horizontal', shrink=0.8, aspect=30, pad=0.08, label='Log10 Number of Data Points'); 
fig.colorbar(image, ax=ax[1], orientation='horizontal', shrink=0.8, aspect=30, pad=0.08, label='melt rate [m/yr]');

# metl distribution plots
fig, ax = plt.subplots(1, 2,figsize=np.array([18, 10]))

plt.sca(ax[0]) 
image = plt.pcolormesh(mbins, ubins, np.log10(muhist.statistic), cmap=cmap, shading='flat') 
plt.title('Distribution of u-melt pairs')

plt.sca(ax[1]) 
image = plt.pcolormesh(mbins, dtbins, np.log10(mthist.statistic), cmap=cmap, shading='flat') 
plt.title('Distribution of dT-melt pairs')

plt.subplots_adjust(wspace=0.025)
fig.colorbar(image, ax=ax[0], orientation='horizontal', shrink=0.8, aspect=30, pad=0.08, label='Log10 Number of Data Points'); 
fig.colorbar(image, ax=ax[1], orientation='horizontal', shrink=0.8, aspect=30, pad=0.08, label='Log10 Number of Data Points');

# melt mean plots
fig, ax = plt.subplots(1, 2,figsize=np.array([18, 10]))

plt.sca(ax[0]) 
image = plt.pcolormesh(mbins, ubins, mumean.statistic, cmap=cmap, shading='flat') 
plt.title('mean dT on distribution of u-melt pairs')

plt.sca(ax[1]) 
image = plt.pcolormesh(mbins, dtbins, mtmean.statistic, cmap=cmap, shading='flat') 
plt.title('mean speed on distribution of dT-melt pairs')

plt.subplots_adjust(wspace=0.025)
fig.colorbar(image, ax=ax[0], orientation='horizontal', shrink=0.8, aspect=30, pad=0.08, label='dT [degC]'); 
fig.colorbar(image, ax=ax[1], orientation='horizontal', shrink=0.8, aspect=30, pad=0.08, label='spd [m/s]');

# ut-melt plots
fig, ax = plt.subplots(1, 2,figsize=np.array([18, 10]))

plt.sca(ax[0]) 
image = plt.pcolormesh(utbins, mbins, np.log10(uthist.statistic.T), cmap=cmap, shading='flat') 
plt.title('Distribution of uT-melt pairs')

plt.sca(ax[1]) 
image = plt.pcolormesh(utbins, mbins, uttmean.statistic.T, cmap=cmap, shading='flat') 
plt.title('mean temperature on distribution of uT-melt pairs')

plt.subplots_adjust(wspace=0.025)
fig.colorbar(image, ax=ax[0], orientation='horizontal', shrink=0.8, aspect=30, pad=0.08, label='Log10 Number of Data Points'); 
fig.colorbar(image, ax=ax[1], orientation='horizontal', shrink=0.8, aspect=30, pad=0.08, label='dT [degC]');

In [ ]:

fig, ax = map_setup(figsize,projection) 

ubar = stats.binned_statistic_2d(lat, lon, u, bins=[latbins, lonbins], statistic='mean')  
vbar = stats.binned_statistic_2d(lat, lon, v, bins=[latbins, lonbins], statistic='mean')  
meanflow = np.sqrt(ubar.statistic ** 2 + vbar.statistic ** 2)

image = plt.pcolormesh(lonbins, latbins, meanflow, cmap=cmap, shading='flat',transform=ccrs.PlateCarree()) 
plt.title('Speed of Mean Surface Currents')
plt.clim(0, 80) 

fig.colorbar(image, ax=ax, orientation='vertical', shrink=0.5, aspect=30, pad=0.05, label='Speed (cm/s)'); 